In [1]:
import click
import glob
import logging
import os
import tarfile


from geocovid.constants import DATA_DIR, TMP_DIR
from geocovid.data_pipeline import extract_data_spark, transform_data_spark
from geocovid.model import GeoCovidModel
from geocovid.utils import start_spark

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")

INFO:root:test


In [7]:
files = glob.glob(os.path.join(DATA_DIR, "*.tar.gz"))
sorted_files = sorted(files)

for file in sorted_files:
    with tarfile.open(file, "r:gz") as tfile:
        path = os.path.join(TMP_DIR, os.path.basename(file).split(".")[0])
        try:
            tfile.extractall(path=TMP_DIR, members=tfile)
            logger.info("dirname %s", path)
        except:
            logger.info("FAILED dirname %s", path)
        tfile.close()

INFO:root:dirname /tmp/geocovid/iso_date=2020-06-01
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-02
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-03
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-04
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-05
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-06
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-07
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-08
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-09
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-10
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-11
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-12
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-13
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-14
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-15
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-16
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-17
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-18
INFO:root:dirname /tmp/geocovid/iso_date=2020-06-19
INFO:root:di

In [14]:
#spark = start_spark()
sdf = extract_data_spark(TMP_DIR, spark)

In [15]:
sdf.show(10)

+--------------------+----------+----------+----------+------------+----------+
|                  id| timestamp|  latitude| longitude|  geohash_12|  iso_date|
+--------------------+----------+----------+----------+------------+----------+
|F50B8A545392ABFF4...|1592456953|-34.841015|   -55.989|6cb1vcqh3u45|2020-06-18|
|F9334BAD08B17DC9F...|1592519793|-34.915546|-56.169582|6cb13gfd1ykc|2020-06-18|
|BC0FDB9F2BEA2140F...|1592481848| -34.75533| -56.01496|6cb4m2gpsbfs|2020-06-18|
|394DDABB5244132B4...|1592457048| -33.12643| -58.31124|6dne91e09d59|2020-06-18|
|093A2CBDCC1F0D16A...|1592518731| -34.71241|-55.956306|6cb4w8vce9fq|2020-06-18|
|056F5AF6AEDC28A87...|1592505926|-34.651134|-56.588478|69zfb7cw8bk7|2020-06-18|
|49C56ABAD402DE3B6...|1592446707|-34.813244| -56.22547|6cb1bw3y5t4u|2020-06-18|
|DC27F4E50AE37C401...|1592482385|-31.346367|-57.859383|6dqvems44tt6|2020-06-18|
|859EE2C24A5CAFB35...|1592493144|-34.906605|-56.154625|6cb16jmqq2vn|2020-06-18|
|9D138D3A857CE4CE3...|1592454620| -34.92

In [16]:
sdf.select('id').distinct().count()

33647

In [17]:
sdf.createOrReplaceTempView("points")

In [18]:
spark.sql("select distinct(count(id)) from points").show()

+---------+
|count(id)|
+---------+
| 41226296|
+---------+



In [19]:
point_df = spark.sql(
        """
        SELECT points.id, hour(cast(points.timestamp AS timestamp)) AS h,
                ST_Envelope_Aggr(ST_Point(cast(points.latitude AS Decimal(24,20)),
                                          cast(points.longitude AS Decimal(24,20)))) 
                AS geometry
        FROM points group by h, id
        """
    )

In [20]:
point_df.createOrReplaceTempView("points_agg")

In [21]:
area_df = spark.sql(
    """
    SELECT id, h, ST_Area(points_agg.geometry) as area
    FROM points_agg
    """
)

In [23]:
area_df.describe('h').show()

+-------+-----------------+
|summary|                h|
+-------+-----------------+
|  count|           584223|
|   mean|11.69547415969587|
| stddev|6.918336370774114|
|    min|                0|
|    max|               23|
+-------+-----------------+



In [ ]:
    clean_df = spark.sql(
        """
        SELECT id, h,
            CASE WHEN ST_Area(points_agg.geometry) < 0.0001
            THEN points_agg.geometry
            ELSE ST_Centroid(points_agg.geometry)
            END as geometry
        FROM points_agg
        """
    )